# Prep of Env and Google Drive

In [1]:
#-- connecting to Drive
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#-- setting up env
!pip install torch
!pip install rasterio
!pip install numpy as np
!pip install pandas as pd
!pip install wandb
!pip install torchvision
!pip install opencv-python
!pip install scikit-image
!pip install geopandas
!pip install lpips
!pip install torchgeometry

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [3]:
cd /content/drive/MyDrive/thesis

/content/drive/MyDrive/thesis


# Template Code

## Imports

In [4]:
# imports
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import wandb
from prettytable import PrettyTable

# local imports
from models_parameters import losses
from models_parameters import models
from utils.dataloader_SRCNN import Dataset as Dataset_interpolate
from utils.dataloader import Dataset as Dataset
from utils.dataloader_downsampledSpot import Dataset as Dataset_downsampledSpot
from utils import helper_functions

## Create DataLoader Object

In [5]:
# inputs
spot6_mosaic = "/home/simon/CDE_UBS/thesis/data_collection/spot6/spot6_mosaic.tif"
spot6_path = "/content/drive/MyDrive/thesis/data/spot6/"
sen2_path = "/content/drive/MyDrive/thesis/data/sen2/"
closest_dates_filepath = "/home/simon/CDE_UBS/thesis/data_loader/data/closest_dates.pkl"

In [6]:
# get dataset object
dataset = Dataset_interpolate(spot6_mosaic,sen2_path,spot6_path,closest_dates_filepath,window_size=500,factor=(10/1.5))
# create dataloader object
loader = DataLoader(dataset,batch_size=1, shuffle=True, num_workers=2)
print("Loader Length: ",len(loader))

precalculated dataset file found, loading...

Dataset Object successfully instanciated!
Loader Length:  68150


## Prep Env and Load Model

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
model = models.SRCNN()
loss_func = losses.loss_psnr

In [9]:
helper_functions.count_parameters(model)

+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
| conv1.weight |   15552    |
|  conv1.bias  |     64     |
| conv2.weight |   51200    |
|  conv2.bias  |     32     |
| conv3.weight |    2400    |
|  conv3.bias  |     3      |
+--------------+------------+
Total Trainable Params: 69251


## Train function

In [10]:
# implementation of model trainer function
def train_model(model,batch_size=1,lr=0.01,epochs=10,wandb_name="test"):
    
    logging=True
    plotting=True # set if images should be plotted
    plot_frequency=10 # set after how manny batches images shoud be plotted

    if logging==True:
        wandb.init(project=wandb_name, entity="simon-donike")
        wandb.config = {
          "learning_rate": lr,
          "epochs": epochs,
          "batch_size": batch_size
        }
    
    # define loaders
    loader_train = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=1)
    loader_test  = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=1)
    loader_full  = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=1)


    train_loss = []  # where we keep track of the training loss
    train_accuracy = []  # where we keep track of the training accuracy of the model
    val_loss = []  # where we keep track of the validation loss
    val_accuracy = []  # where we keep track of the validation accuracy of the model
    epochs = epochs  # number of epochs

    # initialize model

    model = model.float()
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for e in range(epochs):
        model.train()
        train_correct = 0
        batch_count=0
        for (x_train_batch, y_train_batch) in loader_train:
            batch_count = batch_count+1
            x_train_batch = x_train_batch.to(torch.float)
            x_train_batch = x_train_batch.to(device)

            y_train_batch = y_train_batch.to(torch.float) 
            y_train_batch = y_train_batch.to(device)
            y_hat = model(x_train_batch)  # forward pass

            loss = loss_func(y_hat, y_train_batch)  # compute the loss

            loss.backward()  # obtain the gradients with respect to the loss
            optimizer.step()  # perform one step of gradient descent
            optimizer.zero_grad()  # reset the gradients to 0
            y_hat_class = torch.argmax(y_hat.detach(), axis=1)  # we assign an appropriate label based on the network's prediction
            train_correct += torch.sum(y_hat_class == y_train_batch)
            train_loss.append(loss.item() / len(x_train_batch))
            
            if logging==True:
                wandb.log({'loss': loss.item() / len(x_train_batch)})
            if plotting==True:
                if batch_count%plot_frequency==0:
                    print("epoch: ",str(epochs),"batch count: ",batch_count,"            ")# ,end="\r"
                    helper_functions.plot_tensors(y_train_batch,x_train_batch,y_hat,loss)

        train_accuracy.append(train_correct / len(loader_train.dataset))
        
        #print('train_acc: ', train_correct / len(loader_train.dataset))
        if logging==True:
            wandb.log({'train_acc': train_correct / len(loader_train.dataset)})


        print ('Epoch', e+1, ' finished.')

    if logging==True:
        wandb.finish()

# Perform Training

In [11]:
train_model(model=model,batch_size=1,lr=0.01,epochs=10,wandb_name="SRCNN")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
 # work with cuda memory if OOM

 torch.cuda.empty_cache()
 torch.cuda.memory_summary(device=None, abbreviated=False)